In [25]:
import Oanda
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [26]:
oanda = Oanda.Oanda()
today = date.today()
startDate = today - timedelta(days=365*4)
spx_data = oanda.getHistoricalData("SPX500_USD",startDate, today,"D" )
btc_data = oanda.getHistoricalData("BTC_USD",startDate, today,"D" )

<Response [200]>
<Response [200]>


In [27]:
spx_data

,close,high,low,open,Year,Month,Day,Hour,Minute
0,2783.6,2785.2,2742.2,2751.0,2018,2,25,22,0
1,2745.8,2789.1,2741.6,2783.6,2018,2,26,22,0
2,2718.6,2761.2,2711.8,2746.6,2018,2,27,22,0
3,2679.0,2730.5,2658.2,2718.4,2018,2,28,22,0
4,2690.0,2695.6,2647.0,2678.9,2018,3,1,22,0
...,...,...,...,...,...,...,...,...,...
1029,4346.9,4414.4,4324.2,4376.9,2022,2,17,22,0
1030,4292.8,4394.1,4288.4,4332.3,2022,2,20,22,0
1031,4315.0,4361.6,4253.0,4269.5,2022,2,21,22,0
1032,4218.0,4348.5,4215.5,4315.5,2022,2,22,22,0


In [28]:
spx_data["Daily_Return"] = spx_data["close"].pct_change
btc_data["Daily_Return"] = btc_data["close"].pct_change

In [34]:
def createFeatures(data:pd.DataFrame) -> pd.DataFrame:
    data["Daily_Return"] = (data["close"] - data["open"].shift(1))/data["open"].shift(1)
    data["Daily_candel"] = (data["close"] - data["open"])/data["open"]
    data["log_close"] = np.log(1+(data["close"]-data["close"].shift(1))/data["close"].shift(1))
    

In [35]:
createFeatures(spx_data)
createFeatures(btc_data)

In [36]:
spx_data

,close,high,low,open,Year,Month,Day,Hour,Minute,Daily_Return,Daily_candel,log_close
0,2783.6,2785.2,2742.2,2751.0,2018,2,25,22,0,NaN,0.011850,NaN
1,2745.8,2789.1,2741.6,2783.6,2018,2,26,22,0,-0.001890,-0.013580,-0.013673
2,2718.6,2761.2,2711.8,2746.6,2018,2,27,22,0,-0.023351,-0.010194,-0.009955
3,2679.0,2730.5,2658.2,2718.4,2018,2,28,22,0,-0.024612,-0.014494,-0.014673
4,2690.0,2695.6,2647.0,2678.9,2018,3,1,22,0,-0.010447,0.004143,0.004098
...,...,...,...,...,...,...,...,...,...,...,...,...
1029,4346.9,4414.4,4324.2,4376.9,2022,2,17,22,0,-0.027409,-0.006854,-0.007220
1030,4292.8,4394.1,4288.4,4332.3,2022,2,20,22,0,-0.019215,-0.009118,-0.012524
1031,4315.0,4361.6,4253.0,4269.5,2022,2,21,22,0,-0.003993,0.010657,0.005158
1032,4218.0,4348.5,4215.5,4315.5,2022,2,22,22,0,-0.012062,-0.022593,-0.022736
